In [ ]:
# LSTM for Stock Market Prediction with Gann Features (multiple tickers, no scaling)

import math
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

np.random.seed(42)
tf.random.set_seed(42)

# ----------------- Config -----------------
TICKERS = ["AAPL", "MSFT", "GOOG"]   # List of tickers
START = "2015-01-01"
END = "2025-01-01"
SEQ_LEN = 21
EPOCHS = 30
BATCH_SIZE = 32
# ------------------------------------------

import numpy as np

def find_swing_points(df, window=15):
    """
    Identify swing highs and lows using only past data.
    A swing high is the highest point in the past 'window' days,
    a swing low is the lowest point in the past 'window' days.
    """
    swing_highs, swing_lows = [], []
    for i in range(window, len(df)):
        high = df['High'].iloc[i]
        low = df['Low'].iloc[i]
        if high == df['High'].iloc[i-window:i+1].max():
            swing_highs.append((df.index[i], high))
        if low == df['Low'].iloc[i-window:i+1].min():
            swing_lows.append((df.index[i], low))
    return swing_highs, swing_lows


def add_gann_angle_features(df, swing_window=15, angles=None):
    """
    Add pivot-based Gann Angle features (distance + above/below).
    Uses past-only swing detection (no leakage).
    """
    if angles is None:
        angles = {"1x1": 1.0, "2x1": 2.0, "1x2": 0.5}

    df = df.copy()
    swing_highs, swing_lows = find_swing_points(df, swing_window)
    pivots = sorted(swing_highs + swing_lows, key=lambda x: x[0])

    # Init columns
    for name in angles.keys():
        df[f"Angle_{name}_Dist"] = np.nan
        df[f"Angle_{name}_Above"] = np.nan

    # Assign features pivot-to-pivot
    for i, (date, price) in enumerate(pivots):
        direction = 1 if (date, price) in swing_lows else -1
        if i < len(pivots) - 1:
            next_date = pivots[i+1][0]
            mask = (df.index >= date) & (df.index < next_date)
        else:
            mask = (df.index >= date)

        days = np.arange(mask.sum())
        for name, slope in angles.items():
            projected = price + direction * slope * days
            actual = df.loc[mask, 'Close'].values
            dist = actual - projected
            above = (actual > projected).astype(int)

            df.loc[mask, f"Angle_{name}_Dist"] = dist
            df.loc[mask, f"Angle_{name}_Above"] = above

    df.fillna(method="ffill", inplace=True)
    df.fillna(method="bfill", inplace=True)
    return df


def create_sequences(arr, seq_len, target_col_idx=3):
    X, y = [], []
    for i in range(len(arr) - seq_len):
        X.append(arr[i:i+seq_len])
        y.append(arr[i+seq_len, target_col_idx])
    return np.array(X), np.array(y)

def build_lstm(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=False, input_shape=input_shape),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

for ticker in TICKERS:
    print(f"\n===== Running LSTM + Gann for {ticker} =====")

    # 1) Download data
    df = yf.download(ticker, start=START, end=END, progress=False)
    df = df[['Open', 'High', 'Low', 'Close']].round(2).dropna()

    # 2) Add Gann features
    df = add_gann_angle_features(df)


    # 3) Sequence creation
    values = df.values
    X, y = create_sequences(values, SEQ_LEN, 3)

    # Train-test split
    split = int(len(X) * 0.8)
    X_train, y_train = X[:split], y[:split]
    X_test, y_test = X[split:], y[split:]

    # 4) Model
    model = build_lstm((SEQ_LEN, X.shape[2]))
    model.fit(X_train, y_train, validation_data=(X_test, y_test),
              epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)

    # 5) Evaluate
    pred = model.predict(X_test).ravel()
    rmse = math.sqrt(mean_squared_error(y_test, pred))
    mae = mean_absolute_error(y_test, pred)
    mape = mean_absolute_percentage_error(y_test, pred)

    print(f"{ticker} → RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.4f}")


===== Running LSTM + Gann for AAPL =====


/tmp/ipython-input-984675105.py:104: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().